# Реализуйте задания

1. Расширте класс для работы с матрицами, загрузкой и сохранением. Придумаейте тесты.

In [4]:
import pickle
import numpy as np

class PickleHandler:
    @staticmethod
    def save_to_file(obj, filename):
        """Сохраняет объект в файл с помощью pickle."""
        with open(filename, 'wb') as file:
            pickle.dump(obj, file)
    
    @staticmethod
    def load_from_file(filename):
        """Загружает объект из файла с помощью pickle."""
        with open(filename, 'rb') as file:
            return pickle.load(file)

class Matrix:
    def __init__(self, rows):
        self.rows = rows
        self.num_rows = len(rows)
        self.num_cols = len(rows[0]) if rows else 0

    def __add__(self, other):
        """сложение матриц"""
        if self.num_rows != other.num_rows or self.num_cols != other.num_cols:
            raise ValueError("Матрицы должны быть одного размера")
        result = []
        for i in range(self.num_rows):
            row = []
            for j in range(self.num_cols):
                row.append(self.rows[i][j] + other.rows[i][j])
            result.append(row)
        return Matrix(result)

    def __sub__(self, other):
        """вычитание матриц"""
        if self.num_rows != other.num_rows or self.num_cols != other.num_cols:
            raise ValueError("Матрицы должны быть одного размера")
        result = []
        for i in range(self.num_rows):
            row = []
            for j in range(self.num_cols):
                row.append(self.rows[i][j] - other.rows[i][j])
            result.append(row)
        return Matrix(result)

    def __mul__(self, other):   # Cji = Aik * Bkj
        """произведение матриц"""
        if self.num_cols != other.num_rows:
            raise ValueError("Количество столбцов в первой матрице должно быть равно количеству строк во второй матрице.")
        result = []
        for i in range(self.num_rows):
            row = []
            for j in range(other.num_cols):
                element = 0
                for k in range(self.num_cols):
                    element += self.rows[i][k] * other.rows[k][j]
                row.append(element)
            result.append(row)
        return Matrix(result)

    def transpose(self):
        """транспонирование матрицы: - первая строка матрицы А становится первым столбцом матрицы В"""
        transposed = [[self.rows[j][i] for j in range(self.num_rows)] for i in range(self.num_cols)]
        return Matrix(transposed)

    def determinant(self):
        """определитель матрицы, способ рекурсивного поиска"""
        if self.num_rows != self.num_cols:
            raise ValueError("Определитель можно вычислить только для квадратных матриц.")
        if self.num_rows == 1:
            return self.rows[0][0]
        if self.num_rows == 2:
            return self.rows[0][0] * self.rows[1][1] - self.rows[0][1] * self.rows[1][0]
        det = 0
        for j in range(self.num_cols):
            minor = [row[:j] + row[j + 1:] for row in self.rows[1:]]
            det += ((-1) ** j) * self.rows[0][j] * Matrix(minor).determinant()
        return det

    def inverse(self):
        """Вычисление обратной матрицы с помощью NumPy"""
        if self.num_rows != self.num_cols:
            raise ValueError("Можно вычислить только для квадратных матриц")

        # Преобразуем список списков в массив NumPy
        matrix_np = np.array(self.rows)

        # Проверяем, является ли матрица сингулярной
        if np.linalg.det(matrix_np) == 0:
            raise ValueError("Матрица сингулярна, ее нельзя вычислить обратно")

        # Вычисляем обратную матрицу с помощью NumPy
        inverse_matrix_np = np.linalg.inv(matrix_np)

        # Преобразуем обратную матрицу из массива NumPy обратно в список списков
        inverse_matrix = inverse_matrix_np.tolist()
        return Matrix(inverse_matrix)
  

In [5]:
original_matrix = Matrix([[1, 2], [3, 4]])
PickleHandler.save_to_file(original_matrix, "matrix1.pkl")
loaded_matrix = PickleHandler.load_from_file("matrix1.pkl")
assert loaded_matrix.rows == original_matrix.rows


original_matrix = Matrix([])
PickleHandler.save_to_file(original_matrix, "matrix2.pkl")
loaded_matrix = PickleHandler.load_from_file("matrix2.pkl")
assert loaded_matrix.rows == original_matrix.rows

2. Расширте классы Warehouse Product Order для управления складскими запасами, загрузкой и сохранением. Придумаейте тесты.

In [5]:
class Product:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity

    def __repr__(self):
        return f"Product(name={self.name}, price={self.price}, quantity={self.quantity})"
        
    def __eq__(self, other):
        if isinstance(other, Product):
            return (self.name, self.price, self.quantity) == (other.name, other.price, other.quantity)
            
        return False
        
class Warehouse:
    def __init__(self):
        self.products = []

    def add_product(self, product: Product):
        """метод добавления товара на склад"""
        self.products.append(product)

    def __str__(self):
        """вывод всех товаров на складе"""
        for product in self.products:
            return "\n".join(str(product) for product in self.products)

    def get_product(self, product_name):
        """Метод для получения продукта по имени"""
        for product in self.products:
            if product.name == product_name:
                return product
        return None

class Order:
    def __init__(self):
        self.items = []

    def add_item(self, product_name: str, quantity: int, warehouse: Warehouse):
        """Метод добавления товара в корзину с учетом товаров на складе"""
        product = warehouse.get_product(product_name)
        if product and product.quantity >= quantity:
            self.items.append((product_name, quantity))
            return True
        return False

    def buy_item(self, warehouse: Warehouse):
        """Метод покупки товара со склада"""
        for product_name, quantity in self.items:
            product = warehouse.get_product(product_name)
            if product:
                product.quantity -= quantity
            return True
        return False


In [6]:
ball = Product("ball", 1000, 35)
sport_master = Warehouse()
sport_master.add_product(ball)

order = Order()
order.add_item("ball", 2, sport_master)
order.buy_item(sport_master)

PickleHandler.save_to_file(sport_master, "warehouse.pkl")
load_sport_master = PickleHandler.load_from_file("warehouse.pkl")

# получение продукта из загруженного склада
loaded_ball = load_sport_master.get_product("ball")
assert loaded_ball == Product("ball", 1000, 33)